지역
- 수원시 팔달구
- B군, 수요 데이터 없음

중심점
- 공영주차장

고려한 요소들
- 쏘카존
- 지하철
- 아파트

스케일링 여부
- apply standardization and linear transformation onto [0,100] of probability 95.45% (2 sigma)

In [1]:
import pandas as pd
import numpy as np
from numpy import sqrt 

import dataframe_image as dfi

In [2]:
# 데이터프레임을 그림 파일로 저장
save_option = 3 # 1 : dfi / 2 : df2img / 3 : do nothing
import df2img
def save(df, path):
    if save_option == 1:
        dfi.export(df, path)
    if save_option == 2:
        df2img.save_dataframe(fig=df2img.plot_dataframe(df), filename=path)
    if save_option == 3:
        pass

각 군집별 도시들
- cluster A : ['고양시 덕양구', '김포시', '성남시 분당구', '시흥시', '용인시 기흥구', '의정부시', '파주시']
- cluster B : ['구리시', '군포시', '성남시 수정구', '성남시 중원구', '수원시 장안구', '수원시 팔달구', '안성시', '안양시 만안구', '양주시', '오산시', '용인시 처인구', '의왕시', '이천시']
- cluster C : ['남양주시', '부천시', '화성시']
- cluster D : ['가평군', '과천시', '동두천시', '양평군', '여주시', '연천군', '포천시']
- cluster E : ['고양시 일산동구', '고양시 일산서구', '광명시', '광주시', '수원시 권선구', '수원시 영통구', '안산시 단원구', '안산시 상록구', '안양시 동안구', '용인시 수지구', '하남시']
- cluster F : ['평택시']

각 군집별 (수요 데이터가 있는) 도시들
- Cluster A : ['고양시 덕양구', '김포시', '성남시 분당구', '의정부시']
- Cluster B : ['구리시', '성남시 수정구', '성남시 중원구', '안양시 만안구']
- Cluster C : ['남양주시', '화성시']
- Cluster D : ['양평군']
- Cluster E : ['고양시 일산동구', '고양시 일산서구', '광명시', '안양시 동안구', '하남시']
- Cluster F : []

수원시 팔달구는 B군에 속해있고, 수요데이터는 없는 도시이다.

# 데이터 작업

## 아파트

In [3]:
apartments = pd.read_csv("../../data/data_b_raw/2b216_0205_경기도 수원시_공동주택현황_20220705.csv", encoding='CP949')
apartments.dropna(subset=['세대수'], inplace=True)
apartments = apartments[['아파트명', '소재지도로명주소', '세대수', '경도', '위도']]
apartments.columns = ['아파트 이름', '아파트 주소', '세대 수', 'x', 'y']
apartment_names = apartments[['아파트 이름', '아파트 주소']]
apartment_families = apartments[['세대 수']].astype(int)
apartments.drop(columns=['아파트 이름', '아파트 주소', '세대 수'],inplace=True)
apartments = apartments.values
apartments.shape

(549, 2)

## 공영주차장

In [4]:
parks = pd.read_csv("../../data/data_b_raw/2b208_0202_한국교통안전공단_전국공영주차장정보_20191224.csv", encoding='CP949')
parks.rename(columns={'주차장지번주소':'주차장 주소', '주차장명':'주차장 이름'}, inplace=True)
parks.drop_duplicates(subset=['주차장 이름'], inplace=True)
parks = parks[parks['지역구분']=='경기도'].reset_index(drop=True)
parks = parks[parks['지역구분_sub']=='수원시 팔달구'].reset_index(drop=True)
parks

,주차장관리번호,주차장 이름,경도,위도,주차장구분,주차장유형,주차장 주소,주차장도로명주소,주차구획수,운영요일,...,공휴일운영종료시각,요금정보,관리기관명,지역구분,지역구분_sub,지역중심좌표(X좌표),지역중심좌표(Y좌표),지역코드,연락처,수정일자
0,18753,우만공원공영주차장,127.032528,37.286709,공영,노상,경기도 수원시 팔달구 우만동,경기도 수원시 팔달구 세지로 436,119,평일+토요일+공휴일,...,0:00,유료,수원시시설관리공단,경기도,수원시 팔달구,127.016238,37.277463,41115,031-241-1134,2016-07-01
1,18754,우만공영,127.032478,37.286809,공영,노외,경기도 수원시 팔달구 우만동 500,경기도 수원시 팔달구 세지로 436(우만동),119,평일+토요일+공휴일,...,0:00,유료,경기도 수원시 시설관리공단,경기도,수원시 팔달구,127.016238,37.277463,41115,031-240-2833,2016-06-15
2,18772,인계공영,127.030569,37.274656,공영,노외,경기도 수원시 팔달구 인계동 944,경기도 수원시 팔달구 장다리로 306번길 24,64,평일+일요일,...,17:00,유료,경기도 수원시 시설관리공단,경기도,수원시 팔달구,127.016238,37.277463,41115,031-240-2833,2015-01-23
3,18775,인계공영주차장,127.030528,37.274599,공영,노외,경기도 수원시 팔달구 인계동,경기도 수원시 팔달구 장다리로306번길 24,64,평일,...,0:00,유료,수원시시설관리공단,경기도,수원시 팔달구,127.016238,37.277463,41115,031-222-2384,2016-07-01
4,18776,동서주차장,127.030460,37.260881,민영,노외,경기도 수원시 권선구 권선동 1014-3번지,NaN,11,평일,...,NaN,유료,경기도 수원시 권선구청,경기도,수원시 팔달구,127.016238,37.277463,41115,031-228-6364,2016-06-15
5,18787,장다리길주차장,127.028718,37.274711,민영,노상,경기도 수원시 팔달구 인계동 955-2번지,NaN,467,평일+토요일+공휴일,...,19:00,유료,수원시시설관리공단,경기도,수원시 팔달구,127.016238,37.277463,41115,031-240-2843,2016-07-01
6,18789,수원시청 본관주차장,127.028644,37.263448,공영,부설,경기도 수원시 팔달구 인계동 1111,경기도 수원시 팔달구 효원로 241(인계동),90,평일,...,NaN,유료,경기도 수원시 시설관리공단,경기도,수원시 팔달구,127.016238,37.277463,41115,031-240-2845,2016-06-15
7,18790,수원시청 별관주차장,127.028644,37.263448,공영,부설,경기도 수원시 팔달구 인계동 1111,경기도 수원시 팔달구 효원로 241(인계동),364,평일,...,NaN,유료,경기도 수원시 시설관리공단,경기도,수원시 팔달구,127.016238,37.277463,41115,031-240-2845,2016-06-15
8,18793,시청2부설주차장,127.028630,37.263342,공영,노외,경기도 수원시 팔달구 인계동,경기도 수원시 팔달구 효원로 241,337,평일,...,0:00,유료,수원시시설관리공단,경기도,수원시 팔달구,127.016238,37.277463,41115,031-267-4286,2016-07-01
9,18794,인계1지구노상,127.028584,37.263406,공영,노상,경기도 수원시 팔달구 인계동 수원시청주변,경기도 수원시 팔달구 경수대로 428번길일원,33,평일,...,0:00,유료,경기도 수원시 시설관리공단,경기도,수원시 팔달구,127.016238,37.277463,41115,031-240-2833,2016-06-15


In [5]:
parks['주차장 이름'].unique()

array(['우만공원공영주차장', '우만공영', '인계공영', '인계공영주차장', '동서주차장', '장다리길주차장',
       '수원시청 본관주차장', '수원시청 별관주차장', '시청2부설주차장', '인계1지구노상', '창룡공영',
       '창룡공영주차장', '수원시청 부설주차장', '인계1지구주차장', '창룡문주차장', '인계동공영',
       '인계동주택가주차장', '연무대주차장', '남수동공영', '팔달주차타워', '남수동주차장', '수원화성박물관 주차장',
       '매교동주차장', '교동공영', '남창동공영', '화성행궁주차장', '매교동공영', '장다리길노상',
       '선경도서관주차장', '고등공영', '화서환승', '화서환승주차장', '화산공영', '꽃뫼환승', '꽃뫼환승주차장'],
      dtype=object)

In [6]:
parks = pd.read_csv("../../data/data_b_raw/2b208_0202_한국교통안전공단_전국공영주차장정보_20191224.csv", encoding='CP949')
parks.rename(columns={'주차장지번주소':'주차장 주소', '주차장명':'주차장 이름'}, inplace=True)
parks.drop_duplicates(subset=['주차장 이름'], inplace=True)
parks = parks[parks['지역구분']=='경기도'].reset_index(drop=True)
parks = parks[parks['지역구분_sub']=='수원시 팔달구'].reset_index(drop=True)
park_names = parks[['주차장 이름', '주차장 주소']]
parks = parks[['경도', '위도']].rename(columns={'경도':'x','위도':'y'})
parks = parks.values
parks.shape

(35, 2)

## 쏘카존

In [7]:
# 쏘카존 갯수 데이터(김선중)
# 출처 : https://d3s0tskafalll9.cloudfront.net/media/documents/20210823_socar_zone.csv (going deeper 4-6)
zones = pd.read_csv("../../data/data_b_raw/2b201_1228_20210823_socar_zone.csv")
zones.rename(columns={'zone_name':'존 이름'}, inplace=True)
zones = zones[zones['region2']=='수원시\u3000팔달구'].reset_index(drop=True)
zone_names = zones[['존 이름']]
zones = zones[['lng','lat']]
zones.rename(columns={'lng':'x','lat':'y'}, inplace=True)
zones = zones.values
zones.shape

(20, 2)

## 지하철역

In [12]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="my_request")

#stations = ['수원역', '화서역']
subways = pd.read_csv("../../data/data_b_raw/2b215_0205_수원시팔달구_지하철역.csv")
for i, loc in enumerate(subways['역 주소']):
    coord = geolocator.geocode(loc)[-1]
    subways.loc[i, 'x'] = coord[1]
    subways.loc[i, 'y'] = coord[0]
subway_names = subways[['역 이름']]
subways = subways[['x', 'y']]
subways = subways.values
subways

ValueError: Must have equal len keys and value when setting with an iterable

# 제안

In [ ]:
n_apartments = len(apartments)
n_parks = len(parks)
n_zones = len(zones)
n_subways = len(subways)
print("n_apartments :",n_apartments)
print("n_parks :",n_parks)
print("n_zones :",n_zones)
print("n_subways :",n_subways)

In [ ]:
# euclidean distance
def d(x:np.array, y:np.array) -> float:
    return np.linalg.norm(x-y)
x = np.array([0,3])
y = np.array([4,0])
d(x,y)

## score 설정

\begin{align*}
f(x)&=a_1 f_1(x) + a_2 f_2(x) + a_3 f_3(x)\\
f_1(x)&=(\text{가장 가까운 쏘카존까지의 거리})\\
f_2(x)&=(\text{가장 가까운 지하철역까지의 거리})\\
f_3(x)&=(\text{근처 아파트 및 세대수})\\
&=\sum_{i=1}^3\frac{(\text{반경 }i\text{km 이내의 세대수})}{\log_2(i+1)}
\end{align*}
<!-- 수원시 팔달구의 diameter는 약 6km -->

위 식에서 $x$는 특정한 공영주차장의 위치로서, 두 개의 숫자인 위도와 경도로 이루어진 2차원 벡터이다.
$f(x)$는 해당 공영주차장이 신규 쏘카존으로 삼기 적절한지를 나타내는 지표로서, 그 값이 클수록 신규 쏘카존으로 삼기 적절하다.
$a_i$는 세 개의 요소 $f_i(x)$의 단위가 통일되어 있지 않았기 때문에 필요한 단위 통일을 위한 standardization과, 각 요소들의 중요성을 감안해 곱하는 가중치의 부여를 의미한다.
$f_3(x)$의 아이디어는 추천시스템의 evaluation metric 중 하나인 DCG(discounted cumulative gain)에서 가져왔다.

## score 계산

### 세대 밀집도

In [ ]:
# 1 degree of lattitude = 111.1km
# 1 km = 1/111.1 degree of lattitude = 0.009 degree of lattitude
# 각 아파트 A 마다 A와의 거리가 0.009(1km) 이하인 아파트들의 개수를 구한다.
onekilometer = 0.009

In [ ]:
# distances : the distances of a parking lot and every apartment.
# DCG : discounted cumulative gain for the number of families.
def make_dcg(park):
    distances = np.array([d(apartment, park) for apartment in apartments])
    dcg = 0
    for i in range(1,4):
        dcg += 1/(np.log2(1+i))*apartment_families[distances < i*onekilometer].sum()
    return dcg[0]
make_dcg(parks[0])
apartments_info = np.array([make_dcg(park) for park in parks])
apartments_info

In [ ]:
apartments_info_ = pd.DataFrame({'세대 밀집도':apartments_info})
apartments_info_

### 쏘카존까지의 거리

In [ ]:
# distances : the distances of a parking lot and every zone
# zone_info : the list of (min,index) pair for every parking lot.
zones_info = []
for park in parks:
    distances = [d(park, zone) for zone in zones]
    zones_info.append(np.array([np.min(distances),np.argmin(distances)]))
zones_info = np.array(zones_info)
zones_info.shape

In [ ]:
zones_info_ = pd.DataFrame({'쏘카존 최소거리':zones_info[:,0], 'index':np.round(zones_info[:,1]).astype(int)})
zones_info_['가까운 존 이름'] = zones_info_['index'].replace(zone_names['존 이름'].to_dict())
zones_info_.drop('index', axis=1, inplace=True)
zones_info_

### 지하철 역까지의 거리

In [ ]:
# distances : the distances of a parking lot and every subway
# subway_info : the list of (min,index) pair for every parking lot.
subways_info = []
for park in parks:
    distances = [d(park, subway) for subway in subways]
    subways_info.append(np.array([np.min(distances),np.argmin(distances)]))
subways_info = np.array(subways_info)
subways_info.shape

In [ ]:
subways_info_ = pd.DataFrame({'지하철 최소거리':subways_info[:,0], 'index':np.round(subways_info[:,1]).astype(int)})
subways_info_['가까운 역 이름'] = subways_info_['index'].replace(subway_names['역 이름'].to_dict())
subways_info_.drop('index', axis=1, inplace=True)
subways_info_

In [ ]:
scores_original = pd.concat([park_names, apartments_info_, zones_info_, subways_info_], axis=1)
# means = scores.describe().loc['mean'].values
# scores['스코어'] = 1/means[0]*(-scores['주차장 최소거리']) + 1/means[1]*scores['쏘카존 최소거리'] + 1/means[2]*scores['지하철 최소거리'] + 1/means[3]*scores['근처 아파트 개수']
# scores.insert(0, '행복주택 주소', list(happy_names))
# scores.sort_values(by='스코어', ascending=False, inplace=True)
scores_original.head()

In [ ]:
scores = scores_original[['세대 밀집도', '쏘카존 최소거리', '지하철 최소거리']]
scores.head()

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scores_standardized = scaler.fit_transform(scores)
scores_standardized = pd.DataFrame(scores_standardized, index=scores.index, columns=scores.columns)
scores_standardized.head()

A linear map $g$ of $[-2, 2]$ onto $\left[0,\frac{100}3\right]$ is such that
\begin{align*}
g(x)&=ax+b\\
g(-2)&=0\\
g(2)&=\frac{100}3
\end{align*}
This gives $a=\frac{25}3$, $b=\frac{50}3$.
Thus, the function $g$ is defined by $x\mapsto\frac{25}3x+\frac{50}3$.

This computation is false by the following reason.

Suppose that $X_1$, $X_2$, $X_3$ are random variables which are equally distributed for $n$ numbers.
For example, for real numbers $x_{1,1}$, $x_{1,2}$, $\cdots$, $x_{1,n}$, $\mathbb P(X_1=x_{1,r})=\frac1n$ for every $1\le r\le n$, and the same is true for $X_2$ and $X_3$.
The standardization transformed $X_i$ into $Z_i$ so that $\mathbb E[Z_i]=0$ and $\sigma(Z_i)=0$, for each $i$.
The goal is to find a linear transformation $T$ so that $Y=T(Z_1,Z_2,Z_3)$ and $\mathbb P(0\le Y\le100)=0.9545$, where $\mathbb P(-2<Z<2)=0.9545$.
More precisely, this is to find $a$ such that $Y=a(Z_1+Z_2+Z_3)+b$.

By a simple grasp, $\mathbb E(Y)=50$ and $\sigma(Y)=25$, so that $b=\mathbb E(b)=\mathbb E[Y]=50$.
Next, $25=\sigma(Y)=a\sigma(Z_1+Z_2+Z_3)$ or
$$a^2V(Z_1+Z_2+Z_3)=625.$$
Assuming that $X_1$, $X_2$, $X_3$ are independent, then $Z_1$, $Z_2$, $Z_3$ are not only independent but also identically distributed.
Therefore
$$a^2=a^2V(Z_1)=\frac{625}3.$$
and $a=\frac{25}{\sqrt{3}}$.

Summing up, $Y=T(Z_1,Z_2,Z_3)=\frac{25}{\sqrt3}(Z_1+Z_2+Z_3)+50$.
We can think of this transformation $T$ as three sub-transformations so that $Y_i=\frac{25}{\sqrt3}Z_i+\frac{50}3$ and $Y=Y_1+Y_2+Y_3$.

**Conclusion :** $x\mapsto\frac{25}{\sqrt 3}x+\frac{50}3$

In [ ]:
# from math import sqrt
# scores_rescaled = scores_standardized.apply(lambda x:(25/sqrt(3))*x+50/3)
# scores_rescaled['적합도'] = scores_rescaled['세대 밀집도'] + scores_rescaled['쏘카존 최소거리'] + scores_rescaled['지하철 최소거리']
# scores_rescaled['적합도'].mean()

In [ ]:
# import matplotlib.pyplot as plt
# plt.hist(scores_rescaled['적합도'], bins=range(0,100,10))
# plt.xticks(range(0,100,10))
# plt.show()

In [ ]:
# scores = pd.concat([scores_original[['주차장 이름', '주차장 주소']], scores_rescaled['적합도']], axis=1)
# scores.sort_values(by='적합도', ascending=False, inplace=True)
# scores

In [ ]:
# save(scores, "results/1d202_0205_scores_수원시_팔달구.png")

## 2:2:1

\begin{align*}
Y&=25\left(\sqrt{\frac25}Z_1+\sqrt{\frac25}Z_2+\sqrt{\frac15}Z_3\right)+50\\
Y_1&=5\sqrt{10}Z_1+\frac{50}3\\
Y_2&=5\sqrt{10}Z_2+\frac{50}3\\
Y_3&=5\sqrt{5}Z_3+\frac{50}3
\end{align*}

In [ ]:
# scores_rescaled = pd.DataFrame()
# scores_rescaled[['세대 밀집도', '쏘카존 최소거리']] = scores_standardized[['세대 밀집도', '쏘카존 최소거리']].apply(lambda x:(5*sqrt(10))*x+50/3)
# scores_rescaled['지하철 최소거리'] = scores_standardized['지하철 최소거리'].apply(lambda x:(5*sqrt(5))*x+50/3)
# scores_rescaled['적합도'] = scores_rescaled['세대 밀집도'] + scores_rescaled['쏘카존 최소거리'] + scores_rescaled['지하철 최소거리']
# scores_rescaled['적합도'].mean()

In [ ]:
# plt.hist(scores_rescaled['적합도'], bins=range(0,100,10))
# plt.xticks(range(0,100,10))
# plt.show()

In [ ]:
# scores = pd.concat([scores_original[['주차장 이름', '주차장 주소']], scores_rescaled['적합도']], axis=1)
# scores.sort_values(by='적합도', ascending=False, inplace=True)
# scores.head()

## a:b:c

\begin{align*}
Y&=25\left(\sqrt{\frac{a}{a+b+c}}Z_1+\sqrt{\frac{b}{a+b+c}}Z_2+\sqrt{\frac{c}{a+b+c}}Z_3\right)+50\\
Y_1&=25\sqrt{\frac a{a+b+c}}Z_1+\frac{50}3\\
Y_2&=25\sqrt{\frac b{a+b+c}}Z_2+\frac{50}3\\
Y_3&=25\sqrt{\frac c{a+b+c}}Z_3+\frac{50}3
\end{align*}

In [ ]:
'주차장 이름', '주차장 주소', '세대 밀집도', '쏘카존 최소거리', '가까운 존 이름', '지하철 최소거리', '가까운 역 이름'

In [ ]:
# a:세대밀집도, b:쏘카존 최소거리, c:적합도
def evaluate_score(a:int, b:int, c:int):
    scores_rescaled = pd.DataFrame()
    scores_rescaled['세대 밀집도'] = scores_standardized['세대 밀집도'].apply(lambda x:25*sqrt(a/(a+b+c))*x+50/3)
    scores_rescaled['쏘카존 최소거리'] = scores_standardized['쏘카존 최소거리'].apply(lambda x:25*sqrt(b/(a+b+c))*x+50/3)
    scores_rescaled['지하철 최소거리'] = scores_standardized['지하철 최소거리'].apply(lambda x:25*sqrt(c/(a+b+c))*x+50/3)
    scores_rescaled['적합도'] = scores_rescaled['세대 밀집도'] + scores_rescaled['쏘카존 최소거리'] + scores_rescaled['지하철 최소거리']
    scores = pd.concat([scores_original[['주차장 이름', '주차장 주소', '세대 밀집도', '쏘카존 최소거리', '가까운 존 이름', '지하철 최소거리', '가까운 역 이름']], scores_rescaled['적합도']], axis=1)
    scores.sort_values(by='적합도', ascending=False, inplace=True)
    return scores
    

In [ ]:
scores = evaluate_score(3,3,1)
scores.reset_index()

In [ ]:
# a:세대밀집도, b:쏘카존 최소거리, c:적합도
def evaluate_score(a:int, b:int, c:int):
    scores_rescaled = pd.DataFrame()
    scores_rescaled['세대 밀집도'] = scores_standardized['세대 밀집도'].apply(lambda x:25*sqrt(a/(a+b+c))*x+50/3)
    scores_rescaled['쏘카존 최소거리'] = scores_standardized['쏘카존 최소거리'].apply(lambda x:25*sqrt(b/(a+b+c))*x+50/3)
    scores_rescaled['지하철 최소거리'] = scores_standardized['지하철 최소거리'].apply(lambda x:25*sqrt(c/(a+b+c))*x+50/3)
    scores_rescaled['적합도'] = scores_rescaled['세대 밀집도'] + scores_rescaled['쏘카존 최소거리'] + scores_rescaled['지하철 최소거리']
    scores = pd.concat([scores_original[['주차장 이름', '세대 밀집도', '쏘카존 최소거리', '지하철 최소거리']], scores_rescaled['적합도']], axis=1)
    scores.sort_values(by='적합도', ascending=False, inplace=True)
    return scores
    

In [ ]:
# 남수동 2등, 선경도서관 3등, 남창동 29등
scores = evaluate_score(3,3,1)
scores.reset_index(drop=True)